# 言語処理100本ノック　第5章 係り受け解析

##40の準備

In [1]:
"""ここを参考に準備：https://qiita.com/yamaru/items/48dcc527f433c22e0af9"""

'ここを参考に準備：https://qiita.com/yamaru/items/48dcc527f433c22e0af9'

In [2]:
!wget https://nlp100.github.io/data/ai.ja.zip

--2022-02-03 14:09:20--  https://nlp100.github.io/data/ai.ja.zip
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17516 (17K) [application/zip]
Saving to: ‘ai.ja.zip’

ai.ja.zip           100%[===================>]  17.11K  --.-KB/s    in 0.001s  

2022-02-03 14:09:21 (27.0 MB/s) - ‘ai.ja.zip’ saved [17516/17516]



In [3]:
!unzip ai.ja.zip
#will use ai.ja.txt

Archive:  ai.ja.zip
  inflating: ai.ja.txt               
  inflating: readme.ai.ja.md         


In [ ]:
# MeCabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8

In [ ]:
# CRF++のソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7QVR6VXJ5dWExSTQ"
FILE_NAME = "crfpp.tar.gz"
!wget 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O $FILE_NAME
!tar xvf crfpp.tar.gz
%cd CRF++-0.58
!./configure && make && make install && ldconfig
%cd ..

In [ ]:
# CaboChaのソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7SDd1Q1dUQkZQaUU"
FILE_NAME = "cabocha-0.69.tar.bz2"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt
!tar -xvf cabocha-0.69.tar.bz2
%cd cabocha-0.69
!./configure -with-charset=utf-8 && make && make check && make install && ldconfig
%cd ..

In [7]:
#cabochaでai.ja.txtを係り受け解析した結果をai.ja.parsedに書き込み
!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt 

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

ヒント：https://nlp100.github.io/data/ai.ja.zip からデータをダウンロードする．

In [ ]:
"""
係り受けとは：ある単位同士が結びついて1つの意味を成していること。前のことばが後ろに「係っている」。後のことばが前のことばから「受けている」。
ざっくりとした説明：  http://国語文法.com/%E3%80%8C%E4%BF%82%E3%82%8A%E5%8F%97%E3%81%91%E3%80%8D%E3%81%A8%E3%81%AF.html
3つの係り受け解析：  http://kanji.zinbun.kyoto-u.ac.jp/~yasuoka/publications/kakariuke.html
"""

In [9]:
"""defining Morph class"""
class Morph:
  def __init__(self, surface, base, pos, pos1):
    self.surface = surface
    self.base = base
    self.pos = pos
    self.pos1 = pos1
  def show(self):
    print(f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}")

In [10]:
filename = 'ai.ja.txt.parsed'
morph_sens = list()
tmp_sen = list()
count=0
with open(filename, mode='r') as f:
  for line in f:
    if line!="EOS\n":#文末を表すEOSでなければ＝1文まで(ai.ja.txtは句読点で区切られていないことも笑)
      if line[0]!="*":#係り受け解析結果は無視
        sp = line.split()#\tでsplitし、末尾の\nを除去
        m = sp[1].split(',')
        a = Morph(sp[0], m[6], m[0], m[1])
        tmp_sen.append(a)
    elif tmp_sen:
        morph_sens.append(tmp_sen)
        tmp_sen = list()


In [11]:
morph_sens
"""
for m in morph_sens[:2]:
  for i in m:
    i.show()
"""

'\nfor m in morph_sens[:2]:\n  for i in m:\n    i.show()\n'

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [12]:
"""文節を表すクラスChunk"""
class Chunk:
  def __init__(self, morphs:list, dst:int):
    self.morphs = morphs
    self.dst = dst
    self.srcs = list()
  def show(self):
    print('morphs:')
    for morph in self.morphs:
      morph.show()
    print('dst: ', self.dst)
    print('srcs: ', self.srcs)

In [13]:
#cabochaでは、文節間の係り受けを解析している
filename = 'ai.ja.txt.parsed'
sentences = list()
chunk_sen = list()
tmp_chunks = list()
dst = 0
with open(filename, mode='r') as f:
  for line in f:
    if line[0]=="*":#係り受け解析結果
      if tmp_chunks:#１つのchunkが終了
        b = Chunk(tmp_chunks, dst)
        chunk_sen.append(b)
        tmp_chunks = list()
        dst = 0
      ch = line.split()
      dst = int(ch[2].split("D")[0])
    elif line!="EOS\n":#形態素解析結果
      sp = line.split()
      m = sp[1].split(',')
      a = Morph(sp[0], m[6], m[0], m[1])
      tmp_chunks.append(a)
    elif line=="EOS\n":#EOS
      if tmp_chunks:#１つのchunkが終了
          b = Chunk(tmp_chunks, dst)
          chunk_sen.append(b)
          tmp_chunks = list()
          dst = 0
      for i, chunk in enumerate(chunk_sen):
        if dst!=-1:
          chunk_sen[chunk.dst].srcs.append(i)
      sentences.append(chunk_sen)
      chunk_sen = list()

In [14]:
"""確認用"""
for sentence in sentences[:3]:
  for i, chunk in enumerate(sentence):
    print(i)
    chunk.show()
    print('-----')

0
morphs:
surface: 人工, base: 人工, pos: 名詞, pos1: 一般
surface: 知能, base: 知能, pos: 名詞, pos1: 一般
dst:  -1
srcs:  [0]
-----
0
morphs:
surface: 人工, base: 人工, pos: 名詞, pos1: 一般
surface: 知能, base: 知能, pos: 名詞, pos1: 一般
dst:  17
srcs:  []
-----
1
morphs:
surface: （, base: （, pos: 記号, pos1: 括弧開
surface: じん, base: じん, pos: 名詞, pos1: 一般
surface: こうち, base: こうち, pos: 名詞, pos1: 一般
surface: のう, base: のう, pos: 助詞, pos1: 終助詞
surface: 、, base: 、, pos: 記号, pos1: 読点
surface: 、, base: 、, pos: 記号, pos1: 読点
dst:  17
srcs:  []
-----
2
morphs:
surface: AI, base: *, pos: 名詞, pos1: 一般
dst:  3
srcs:  []
-----
3
morphs:
surface: 〈, base: 〈, pos: 記号, pos1: 括弧開
surface: エーアイ, base: *, pos: 名詞, pos1: 固有名詞
surface: 〉, base: 〉, pos: 記号, pos1: 括弧閉
surface: ）, base: ）, pos: 記号, pos1: 括弧閉
surface: と, base: と, pos: 助詞, pos1: 格助詞
surface: は, base: は, pos: 助詞, pos1: 係助詞
surface: 、, base: 、, pos: 記号, pos1: 読点
dst:  17
srcs:  [2]
-----
4
morphs:
surface: 「, base: 「, pos: 記号, pos1: 括弧開
surface: 『, base: 『, pos: 記号, pos1: 括弧開
sur

## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [15]:
for sentence in sentences[:3]:#sentences[:3]　最初の3文まで
  for chunk in sentence:
    for morph in chunk.morphs:#係り元の文節を表示
      if morph.pos!="記号":
        print(morph.surface, end='')
    print('\t', end='')
    for morph in sentence[chunk.dst].morphs:#係り先の文節を表示
      if morph.pos!="記号":
        print(morph.surface, end='')
    print('')

人工知能	人工知能
人工知能	語
じんこうちのう	語
AI	エーアイとは
エーアイとは	語
計算	という
という	道具を
概念と	道具を
コンピュータ	という
という	道具を
道具を	用いて
用いて	研究する
知能を	研究する
研究する	計算機科学
計算機科学	の
の	一分野を
一分野を	指す
指す	語
語	研究分野とも
言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術または
技術または	研究分野とも
計算機	コンピュータによる
コンピュータによる	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野とも
研究分野とも	される
される	される


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [16]:
tmp_pair = str()
for sentence in sentences[:3]:
  for chunk in sentence:
    pos_src = list()#係り元の品詞を格納
    sur_src = list()#      の表層系
    pos_dst = list()#係り先の品詞
    sur_dst = list()#      の表層系
      
    for morph in chunk.morphs:#係り元の文節を表示
      if morph.pos!="記号":
        pos_src.append(morph.pos)
        sur_src.append(morph.surface)
    for morph in sentence[chunk.dst].morphs:#係り先の文節を表示
      if morph.pos!="記号":
        pos_dst.append(morph.pos)
        sur_dst.append(morph.surface)
    if '名詞' in pos_src and '動詞' in pos_dst:
      print(''.join(sur_src), '\t', end='')
      print(''.join(sur_dst))


道具を 	用いて
知能を 	研究する
一分野を 	指す
知的行動を 	代わって
人間に 	代わって
コンピューターに 	行わせる
研究分野とも 	される


## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [17]:
!pip install graphviz

In [ ]:
from graphviz import Digraph #有向グラフ

dot = Digraph(format='png')

sentence = sentences[2]
for i, chunk in enumerate(sentence):
  dot.node(str(i))
  if chunk.dst != -1:
    dot.edge(str(i), str(chunk.dst))

dot

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．


```
作り出す    で は を
```

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．  

- コーパス中で頻出する述語と格パターンの組み合わせ
- 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）


In [19]:
pat = list()

for sentence in sentences:
  for chunk in sentence:
    for n in chunk.morphs:
      if n.pos=="動詞":
        jyosi=list()
        for chunknum in chunk.srcs:
          for m in sentence[chunknum].morphs:
            if m.pos=="助詞":
              jyosi.append(m.base)
        if jyosi:#係り元に助詞がある場合
          pat.append([n.base, ' '.join(jyosi)])#動詞、助詞ペアをpatにappend

filename = 'kakupattern.txt'
with open(filename,'w') as f:
  for p in pat:
    f.write('{}\t{}\n'.format(p[0], p[1]))

In [20]:
# UNIXコマンドの検索＆説明　https://www.tutorialspoint.com/unix_commands/
!cat ./kakupattern.txt | sort | uniq -c | sort -nr | head -n 10

    184 する	を
    137 する	て を
     81 する	が は が
     57 ある	は て は が
     24 いる	で は の に て
     13 れる	として は も
      9 行う	を
      9 なる	から で と
      9 られる	が
      8 する	と まで と


In [21]:
!cat ./kakupattern.txt | grep '行う' | sort | uniq -c | sort -nr | head -n 5

      9 行う	を
      5 行う	を に を
      3 行う	て に
      2 行う	で は て が に
      1 行う	まで を に


## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

```
作り出す	で は を	会議で ジョンマッカーシーは 用語を
```

In [24]:
pat = list()

for sentence in sentences:
  for chunk in sentence:
    for n in chunk.morphs:
      if n.pos=="動詞":
        jyosi = list()
        kou = list()
        for chunknum in chunk.srcs:
          for m in sentence[chunknum].morphs:
            if m.pos=="助詞":
              jyosi.append(m.base)
              kou.append(''.join([m.surface for m in sentence[chunknum].morphs]))
        if jyosi:#係り元に助詞がある場合
          pat.append([n.base, ' '.join(jyosi), ' '.join(kou)])#動詞、助詞ペアをpatにappend

filename = '46.txt'
with open(filename,'w') as f:
  for p in pat:
    f.write('{}\t{}\t{}\n'.format(p[0], p[1], p[2]))

## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．

```
学習を行う	に を	元に 経験を
```

In [ ]:
"""
サ変名詞（さへんめいし）https://ja.wiktionary.org/wiki/%E3%82%B5%E5%A4%89%E5%90%8D%E8%A9%9E

(日本語文法) 動詞「する」に接続してサ行変格活用の動詞となりうる名詞のこと。
「該当（する）」、「エンジョイ（する）」など、動作を表わす漢語や名詞化した動詞の外来語などがこれにあたる。
"""

In [29]:
pat = list()

for sentence in sentences:
  for chunk in sentence:#それぞれの文節で
    for n in chunk.morphs:#文節の形態素結果をみる
      if n.pos=="動詞":
        jyosi = list()
        kou = list()
        marumaruwo = str()
        for chunknum in chunk.srcs:#この文節に係っている文節
          for m in sentence[chunknum].morphs:
            if m.pos=="助詞":
              jyosi.append(m.base)
              if m.base=="を" and "サ変接続" in [l.pos1 for l in sentence[chunknum].morphs]:
                  marumaruwo+=(''.join([l.surface for l in sentence[chunknum].morphs]))
                  continue
              kou.append(''.join([m.base for m in sentence[chunknum].morphs]))
        if marumaruwo:#係り元に助詞がある場合
          pat.append([marumaruwo+n.base, ' '.join(jyosi), ' '.join(kou)])#動詞、助詞ペアをpatにappend

filename = '47.txt'
with open(filename,'w') as f:
  for p in pat:
    f.write('{}\t{}\t{}\n'.format(p[0], p[1], p[2]))

## 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
ジョンマッカーシーは -> 作り出した
AIに関する -> 最初の -> 会議で -> 作り出した
最初の -> 会議で -> 作り出した
会議で -> 作り出した
人工知能という -> 用語を -> 作り出した
用語を -> 作り出した
```
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．

```
ジョンマッカーシーは -> 作り出した
ＡＩに -> 関する -> 会議で -> 作り出した
会議で -> 作り出した
人工知能と -> いう -> 用語を -> 作り出した
用語を -> 作り出した
```

In [42]:
for sentence in sentences[5:6]:
  for i, chunk in enumerate(sentence):#それぞれの文節で
    if "名詞" in [l.pos for l in chunk.morphs]:#"名詞"を含む場合
      curdst = chunk.dst
      curchunk = i
      print(''.join([m.surface for m in sentence[curchunk].morphs]), end=' ')
      curchunk = sentence[curchunk].dst
      while sentence[curchunk].dst>0:
        print('->', ''.join([m.surface for m in sentence[curchunk].morphs]), end=' ')
        curchunk = sentence[curchunk].dst
      print('\n')

人間の -> 知的能力を -> 実現する、 -> 技術・ソフトウェア・コンピュータシステム。 -> ある。 -> 命名された。 -> 使われている。 

知的能力を -> 実現する、 -> 技術・ソフトウェア・コンピュータシステム。 -> ある。 -> 命名された。 -> 使われている。 

コンピュータ上で -> 実現する、 -> 技術・ソフトウェア・コンピュータシステム。 -> ある。 -> 命名された。 -> 使われている。 

実現する、 -> 技術・ソフトウェア・コンピュータシステム。 -> ある。 -> 命名された。 -> 使われている。 

様々な -> 技術・ソフトウェア・コンピュータシステム。 -> ある。 -> 命名された。 -> 使われている。 

技術・ソフトウェア・コンピュータシステム。 -> ある。 -> 命名された。 -> 使われている。 

応用例は -> ある。 -> 命名された。 -> 使われている。 

自然言語処理 -> （機械翻訳・かな漢字変換・構文解析等）、 -> 専門家の -> 推論・判断を -> 模倣する -> エキスパートシステム、 -> 画像認識等が -> ある。 -> 命名された。 -> 使われている。 

（機械翻訳・かな漢字変換・構文解析等）、 -> 専門家の -> 推論・判断を -> 模倣する -> エキスパートシステム、 -> 画像認識等が -> ある。 -> 命名された。 -> 使われている。 

専門家の -> 推論・判断を -> 模倣する -> エキスパートシステム、 -> 画像認識等が -> ある。 -> 命名された。 -> 使われている。 

推論・判断を -> 模倣する -> エキスパートシステム、 -> 画像認識等が -> ある。 -> 命名された。 -> 使われている。 

模倣する -> エキスパートシステム、 -> 画像認識等が -> ある。 -> 命名された。 -> 使われている。 

エキスパートシステム、 -> 画像認識等が -> ある。 -> 命名された。 -> 使われている。 

画像データを -> 解析して -> 検出・抽出したりする -> 画像認識等が -> ある。 -> 命名された。 -> 使われている。 

解析して -> 検出・抽出したりする -> 画像認識等が -

## 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号が$i$と$j$（$i$<$j$）のとき，係り受けパスは以下の仕様を満たすものとする．


- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
- 文節$i$と$j$に含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節$i$から構文木の根に至る経路上に文節$j$が存在する場合: 文節$i$から文節$j$のパスを表示
- 上記以外で，文節$i$と文節$j$から構文木の根に至る経路上で共通の文節$k$で交わる場合: 文節$i$から文節$k$に至る直前のパスと文節$j$から文節$k$に至る直前までのパス，文節$k$の内容を” | “で連結して表示

「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
```
Xは | Yに関する -> 最初の -> 会議で | 作り出した
Xは | Yの -> 会議で | 作り出した
Xは | Yで | 作り出した
Xは | Yという -> 用語を | 作り出した
Xは | Yを | 作り出した
Xに関する -> Yの
Xに関する -> 最初の -> Yで
Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した
Xに関する -> 最初の -> 会議で | Yを | 作り出した
Xの -> Yで
Xの -> 会議で | Yという -> 用語を | 作り出した
Xの -> 会議で | Yを | 作り出した
Xで | Yという -> 用語を | 作り出した
Xで | Yを | 作り出した
Xという -> Yを
```

KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．

```
Xは | Yに -> 関する -> 会議で | 作り出した。
Xは | Yで | 作り出した。
Xは | Yと -> いう -> 用語を | 作り出した。
Xは | Yを | 作り出した。
Xに -> 関する -> Yで
Xに -> 関する -> 会議で | Yと -> いう -> 用語を | 作り出した。
Xに -> 関する -> 会議で | Yを | 作り出した。
Xで | Yと -> いう -> 用語を | 作り出した。
Xで | Yを | 作り出した。
Xと -> いう -> Yを
```

In [45]:
"""pattern 1 done. pattern 2, i dunno how to do it..."""
for sentence in sentences[5:6]:
  for i, chunk in enumerate(sentence):#それぞれの文節で
    if "名詞" in [l.pos for l in chunk.morphs]:#"名詞"を含む場合
      curdst = chunk.dst
      curchunk = i
      prevchunk = int()
      ku=str()
      for m in sentence[curchunk].morphs:
        if m.pos=="名詞":
          ku+='X'
        else:
          ku+=m.surface
      print(ku, end=' ')
      prevchunk = curchunk
      curchunk = sentence[curchunk].dst
      flag=0
      while sentence[curchunk].dst>0:
        ku =str()
        for m in sentence[curchunk].morphs:
          if m.pos=="名詞":
            ku+="Y"
            flag=1
          else:
            ku+=m.surface
        if flag==1:
          print('->', ku, end=' ')
          break
        else:
          print('->', ku , end=' ')
        prevchunk = curchunk
        curchunk = sentence[curchunk].dst
      print('\n')

Xの -> YYを 

XXを -> Yする、 

XXで -> Yする、 

Xする、 -> Y・Y・YY。 

Xな -> Y・Y・YY。 

X・X・XX。 -> ある。 -> Yされた。 

XXは -> ある。 -> Yされた。 

XXX -> （YY・YY・YYY）、 

（XX・XX・XXX）、 -> YYの 

XXの -> Y・Yを 

X・Xを -> Yする 

Xする -> YY、 

XX、 -> YYYが 

XXを -> Yして 

Xして -> Y・Yしたりする 

Xの -> Yを 

Xを -> Y・Yしたりする 

X・Xしたりする -> YYYが 

XXXが -> ある。 -> Yされた。 

XXに -> Yされた。 

XXXで -> Yされた。 

X・Xにより -> Yされた。 

Xされた。 -> 使われている。 

Xでは、 -> 使われている。 

XXを -> 用いた -> Yの 

Xの -> Yを 

Xを -> する -> Yでの 

Xと -> する -> Yでの 

Xや -> Yでの 

Xでの -> Yという 

Xという -> Yでも 

Xでも -> 使われている。 

XXXXXの -> YYや 

XXや -> YYも 

XXの -> YYも 

XXも -> 呼ばれる -> Yも 

Xも 

